<a href="https://colab.research.google.com/github/zaahraa1993/Multimodal-sentiment-analysis/blob/main/bert_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[torch] --upgrade
!pip install accelerate --upgrade
!pip install datasets --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 25.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

In [2]:
!pip install datasets

In [13]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [4]:
# Load the IMDB dataset
dataset = load_dataset('imdb')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
# Split the dataset
train_val_data = dataset['train'].train_test_split(test_size=0.2)
train_data, val_data = train_val_data['train'], train_val_data['test']
test_data = dataset['test']


In [6]:
# Save the splits for RNN
np.save('/content/drive/MyDrive/Colab Notebooks/train_data.npy', train_data['text'])
np.save('/content/drive/MyDrive/Colab Notebooks/train_labels.npy', train_data['label'])
np.save('/content/drive/MyDrive/Colab Notebooks/val_data.npy', val_data['text'])
np.save('/content/drive/MyDrive/Colab Notebooks/val_labels.npy', val_data['label'])
np.save('/content/drive/MyDrive/Colab Notebooks/test_data.npy', test_data['text'])
np.save('/content/drive/MyDrive/Colab Notebooks/test_labels.npy', test_data['label'])

In [7]:
# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Prepare the dataset
class IMDBDatasetBERT(Dataset):
    def __init__(self, data_split):
        self.data = data_split

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item['text']
        encoding = tokenizer(text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
        labels = torch.tensor(1 if item['label'] == 1 else 0)
        encoding['labels'] = labels
        return {key: val.squeeze(0) for key, val in encoding.items()}



In [9]:
# Create datasets
train_dataset_bert = IMDBDatasetBERT(train_data)
val_dataset_bert = IMDBDatasetBERT(val_data)
test_dataset_bert = IMDBDatasetBERT(test_data)

In [10]:
# compute metrics function
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='binary')
    acc = accuracy_score(p.label_ids, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

In [11]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=2,
    load_best_model_at_end=True,
    weight_decay=0.05,
    logging_dir='./logs',
    logging_steps=10,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    save_on_each_node=True,
    learning_rate=2e-5,
    warmup_steps=500,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
# Initialize the Trainer
trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_dataset_bert,
    eval_dataset=val_dataset_bert,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

In [15]:
# Train the model
trainer.train()

# Evaluate the model
predictions_output = trainer.predict(test_dataset_bert)
predictions = predictions_output.predictions
true_labels = predictions_output.label_ids


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.258300,0.239167,0.931400,0.931741,0.938653,0.924931
2,0.211100,0.270379,0.933600,0.935182,0.924354,0.946266
3,0.143900,0.334390,0.937000,0.937980,0.934851,0.941130


In [16]:
# Save predictions and labels
bert_predictions = torch.sigmoid(torch.tensor(predictions[:, 1])).numpy()
np.save('bert_predictions.npy', bert_predictions)
np.save('bert_true_labels.npy', true_labels)

# Save the model
bert_model.save_pretrained('bert_model')
tokenizer.save_pretrained('bert_model')

('bert_model/tokenizer_config.json',
 'bert_model/special_tokens_map.json',
 'bert_model/vocab.txt',
 'bert_model/added_tokens.json')